In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D,Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import numpy as np
from sklearn.metrics import accuracy_score
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [5]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):
    old_lr = self.model.optimizer.lr.read_value()
    new_lr = old_lr * 0.99
    print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
    self.model.optimizer.lr.assign(new_lr)
    
def create_checkpoint_callback(model_name):
  return tf.keras.callbacks.ModelCheckpoint(filepath=f"{model_name}/checkpoint.ckpt",
                                                          #  monitor="val_accuracy",
                                                           save_best_only=True,
                                                           save_weights_only=True,
                                                           save_freq="epoch")

es_cb = EarlyStopping(monitor='val_loss', patience=10)

In [3]:
train_dir = "data/train"
test_dir = "data/validation"

In [4]:
data_dir = pathlib.Path(train_dir)
class_names = sorted([item.name for item in data_dir.glob("*")])
print(class_names)

['cats', 'dogs']


In [11]:
IMG_SIZE=(224,224)
IMG_SIZE+(3,)
BATCH_SIZE = 32
EPOCHS = 200
print("Training image:")
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                              image_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE,
                                              label_mode="binary")
print("Testing image:")
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                            image_size=IMG_SIZE,
                                            batch_size=BATCH_SIZE,
                                            label_mode="binary")

Training image:
Found 2000 files belonging to 2 classes.
Testing image:
Found 802 files belonging to 2 classes.


In [19]:
# Create a CNN model (same as Tiny VGG but for binary classification - https://poloclub.github.io/cnn-explainer/ )
model_8 = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)), # same input shape as our images
  layers.Conv2D(10, 3, activation='relu'),
  layers.MaxPool2D(),
  layers.Conv2D(10, 3, activation='relu'),
  layers.Conv2D(10, 3, activation='relu'),
  layers.MaxPool2D(),
  layers.Flatten(),
  layers.Dense(1, activation='sigmoid')
])

# Compile the model
model_8.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_8 = model_8.fit(train_data,
                        epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data,
                        callbacks=[create_checkpoint_callback(model_4.name),es_cb])

Epoch 1/10
63/63 [==============================] - 47s 743ms/step - loss: 0.6812 - accuracy: 0.5460 - val_loss: 0.6598 - val_accuracy: 0.5998
Epoch 2/10
63/63 [==============================] - 46s 733ms/step - loss: 0.6499 - accuracy: 0.5950 - val_loss: 0.6424 - val_accuracy: 0.6072
Epoch 3/10
63/63 [==============================] - 51s 813ms/step - loss: 0.6183 - accuracy: 0.6385 - val_loss: 0.6500 - val_accuracy: 0.6122
Epoch 4/10
63/63 [==============================] - 51s 795ms/step - loss: 0.5847 - accuracy: 0.6865 - val_loss: 0.6699 - val_accuracy: 0.6222
Epoch 5/10
63/63 [==============================] - 55s 881ms/step - loss: 0.5216 - accuracy: 0.7415 - val_loss: 0.7034 - val_accuracy: 0.6185
Epoch 6/10
63/63 [==============================] - 50s 783ms/step - loss: 0.4352 - accuracy: 0.8015 - val_loss: 0.7948 - val_accuracy: 0.6297
Epoch 7/10
63/63 [==============================] - 58s 921ms/step - loss: 0.3639 - accuracy: 0.8395 - val_loss: 0.8676 - val_accuracy: 0.6359

In [5]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [8]:
# Create a CNN model (same as Tiny VGG but for binary classification - https://poloclub.github.io/cnn-explainer/ )
model_8 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(150, 150, 3)), # same input shape as our images
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
],name='img_multiclass_aug')

# Compile the model
model_8.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model
history_8 = model_8.fit(train_generator,
                        epochs=10,
                        validation_data=validation_generator,
                        callbacks=[create_checkpoint_callback(model_8.name),es_cb])

Epoch 1/10
100/100 [==============================] - 27s 265ms/step - loss: 0.6934 - accuracy: 0.5005 - val_loss: 0.6894 - val_accuracy: 0.5000
Epoch 2/10
100/100 [==============================] - 29s 287ms/step - loss: 0.6903 - accuracy: 0.5525 - val_loss: 0.6813 - val_accuracy: 0.5200
Epoch 3/10
100/100 [==============================] - 28s 280ms/step - loss: 0.6903 - accuracy: 0.5305 - val_loss: 0.6866 - val_accuracy: 0.5249
Epoch 4/10
100/100 [==============================] - 27s 274ms/step - loss: 0.6879 - accuracy: 0.5435 - val_loss: 0.6653 - val_accuracy: 0.5686
Epoch 5/10
100/100 [==============================] - 28s 282ms/step - loss: 0.6740 - accuracy: 0.5790 - val_loss: 0.6578 - val_accuracy: 0.5673
Epoch 6/10
100/100 [==============================] - 29s 288ms/step - loss: 0.6731 - accuracy: 0.5720 - val_loss: 0.6555 - val_accuracy: 0.5985
Epoch 7/10
100/100 [==============================] - 31s 309ms/step - loss: 0.6513 - accuracy: 0.6160 - val_loss: 0.6430 - val_ac

In [9]:
model_8.summary()

Model: "img_multiclass_aug"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 148, 148, 10)      280       
                                                                 
 conv2d_9 (Conv2D)           (None, 146, 146, 10)      910       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 73, 73, 10)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 71, 71, 10)        910       
                                                                 
 conv2d_11 (Conv2D)          (None, 69, 69, 10)        910       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 34, 34, 10)       0         
 2D)                                            

In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              metrics=['accuracy'])
history = model.fit(
      train_generator,  # 2000 images = batch_size * steps
      epochs=100,
      validation_data=validation_generator,
      callbacks=[create_checkpoint_callback(model.name),es_cb])

Epoch 1/100
100/100 [==============================] - 39s 388ms/step - loss: 0.6905 - accuracy: 0.5250 - val_loss: 0.6819 - val_accuracy: 0.5786
Epoch 2/100
100/100 [==============================] - 40s 399ms/step - loss: 0.6830 - accuracy: 0.5610 - val_loss: 0.6938 - val_accuracy: 0.5062
Epoch 3/100
100/100 [==============================] - 43s 433ms/step - loss: 0.6770 - accuracy: 0.5720 - val_loss: 0.6575 - val_accuracy: 0.6047
Epoch 4/100
100/100 [==============================] - 42s 422ms/step - loss: 0.6660 - accuracy: 0.5905 - val_loss: 0.6353 - val_accuracy: 0.6372
Epoch 5/100
100/100 [==============================] - 46s 465ms/step - loss: 0.6508 - accuracy: 0.6130 - val_loss: 0.6254 - val_accuracy: 0.6708
Epoch 6/100
100/100 [==============================] - 44s 438ms/step - loss: 0.6383 - accuracy: 0.6300 - val_loss: 0.6598 - val_accuracy: 0.6072
Epoch 7/100
100/100 [==============================] - 40s 401ms/step - loss: 0.6231 - accuracy: 0.6450 - val_loss: 0.6035 -

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [13]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, # Remove dense layer before CONV layers
                                weights = None)

In [14]:
pre_trained_model.trainable=False
pre_trained_model.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
# Choose which layer from the full inception network will be your last layer.
# Here we're choosing mixed7, which outputs a bunch of 7 x 7 convolutions.
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [16]:

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x) # Provide fraction of neurons to drop.                 
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = tf.keras.Model( pre_trained_model.input, x) 

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [17]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
            callbacks=[create_checkpoint_callback(model.name),es_cb]
)

Epoch 1/20
100/100 [==============================] - 73s 708ms/step - loss: 0.3225 - accuracy: 0.8745 - val_loss: 0.1229 - val_accuracy: 0.9501
Epoch 2/20
100/100 [==============================] - 70s 701ms/step - loss: 0.2137 - accuracy: 0.9195 - val_loss: 0.1224 - val_accuracy: 0.9464
Epoch 3/20
100/100 [==============================] - 68s 678ms/step - loss: 0.2096 - accuracy: 0.9180 - val_loss: 0.1077 - val_accuracy: 0.9651
Epoch 4/20
100/100 [==============================] - 68s 680ms/step - loss: 0.1966 - accuracy: 0.9305 - val_loss: 0.1754 - val_accuracy: 0.9526
Epoch 5/20
100/100 [==============================] - 66s 664ms/step - loss: 0.1721 - accuracy: 0.9410 - val_loss: 0.1182 - val_accuracy: 0.9601
Epoch 6/20
100/100 [==============================] - 67s 672ms/step - loss: 0.1774 - accuracy: 0.9360 - val_loss: 0.1108 - val_accuracy: 0.9638
Epoch 7/20
100/100 [==============================] - 66s 658ms/step - loss: 0.1821 - accuracy: 0.9335 - val_loss: 0.1100 - val_ac

# EXAM

In [11]:
fashion_mnist = tf.keras.datasets.fashion_mnist

In [21]:
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [13]:
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [14]:
train_data[0].shape, train_labels[0].shape

((28, 28), ())

In [24]:
# Set random seed
tf.random.set_seed(42)
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
train_data = train_data / 255.0
test_data = test_data / 255.0
# Create the model
model_14 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)), # input layer (we had to reshape 28x28 to 784)
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dense(16, activation="relu"),
  tf.keras.layers.Dense(10, activation="softmax") # output shape is 10, activation is softmax
])

# Compile the model
model_14.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                 optimizer=tf.keras.optimizers.Adam(lr=0.001), # ideal learning rate (same as default)
                 metrics=["accuracy"])

# Fit the model
history = model_14.fit(train_data,
                       train_labels,
                       epochs=20,
                       validation_data=(test_data, test_labels),
                       callbacks=[es_cb,create_checkpoint_callback(model_1.name)]
                      )

Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5605 - accuracy: 0.8054 - val_loss: 0.4533 - val_accuracy: 0.8377
Epoch 2/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4079 - accuracy: 0.8550 - val_loss: 0.4748 - val_accuracy: 0.8415
Epoch 3/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3789 - accuracy: 0.8641 - val_loss: 0.3972 - val_accuracy: 0.8572
Epoch 4/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3553 - accuracy: 0.8728 - val_loss: 0.3936 - val_accuracy: 0.8615
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3411 - accuracy: 0.8778 - val_loss: 0.4118 - val_accuracy: 0.8510
Epoch 6/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3278 - accuracy: 0.8814 - val_loss: 0.3960 - val_accuracy: 0.8558
Epoch 7/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3196 - accuracy: 0.8852 - val_loss: 0.4047 - val_accuracy: